In [4]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from tqdm import tqdm
from fake_useragent import UserAgent

### settings

In [12]:
# init chrome driver
chrome_driver = "C:/Users/seungsu/Desktop/gentlegraph/crawling/chromedriver_win32/chromedriver.exe"

chrome_options = webdriver.ChromeOptions()

chrome_options.add_argument("headless")
chrome_options.add_argument("--window-size=1100,2000")
userAgent = UserAgent().random
chrome_options.add_argument(f'user-agent={userAgent}')

prefs = {'profile.default_content_setting_values': {'cookies' : 2, 'images': 2, 'plugins' : 2, 'popups': 2, 'geolocation': 2, 'notifications' : 2, 'auto_select_certificate': 2, 'fullscreen' : 2, 'mouselock' : 2, 'mixed_script': 2, 'media_stream' : 2, 'media_stream_mic' : 2, 'media_stream_camera': 2, 'protocol_handlers' : 2, 'ppapi_broker' : 2, 'automatic_downloads': 2, 'midi_sysex' : 2, 'push_messaging' : 2, 'ssl_cert_decisions': 2, 'metro_switch_to_desktop' : 2, 'protected_media_identifier': 2, 'app_banner': 2, 'site_engagement' : 2, 'durable_storage' : 2}} 
chrome_options.add_experimental_option('prefs', prefs)
driver = webdriver.Chrome(chrome_driver, options=chrome_options)


### 전체 재생목록 가져오기

In [13]:
import time

driver.get("https://www.youtube.com/playlist?list=PLmtapKaZsgZt3g_uAPJbsMWdkVsznn_2R")
driver.execute_script('window.scrollTo(0, 20000);')
driver.implicitly_wait(2)
playlist = driver.find_elements_by_css_selector("ytd-playlist-video-renderer")

In [14]:
total_contents = []
for content in playlist:
    index = content.find_element_by_css_selector("#index").text
    video_info = content.find_element_by_css_selector("a#video-title")
    channel_info = content.find_element_by_css_selector("ytd-channel-name#channel-name").find_element_by_css_selector("a")
    url = video_info.get_attribute("href")
    title = video_info.text
    creator = channel_info.text
    creator_channel = channel_info.get_attribute("href")
    
    data = {
        "rank" : index,
        "content_url" : url,
        "content_title" : title,
        "creator" : creator,
        "creator_url" : creator_channel
    }
    
    total_contents.append(data)

In [15]:
total_contents

[{'rank': '1',
  'content_url': 'https://www.youtube.com/watch?v=hfXqgH2RscE&list=PLmtapKaZsgZt3g_uAPJbsMWdkVsznn_2R&index=1',
  'content_title': '아이유 모르는 척하기 챌린지 | ODG',
  'creator': 'odg',
  'creator_url': 'https://www.youtube.com/c/ODGkids'},
 {'rank': '2',
  'content_url': 'https://www.youtube.com/watch?v=PXemHTr_xLs&list=PLmtapKaZsgZt3g_uAPJbsMWdkVsznn_2R&index=2',
  'content_title': '[밥묵자] 무서운 우리새끼 (feat. 김종국)',
  'creator': '꼰대희',
  'creator_url': 'https://www.youtube.com/channel/UCSc2DcSzzjHgBn8Hg0KG7PA'},
 {'rank': '3',
  'content_url': 'https://www.youtube.com/watch?v=uJ0J3ZdEMH4&list=PLmtapKaZsgZt3g_uAPJbsMWdkVsznn_2R&index=3',
  'content_title': '[선공개] 브레이브걸스 - 운전만 해♬ 이 노래엔 슬픈 사연이 있어ㅠㅜ #사이코러스 #코미디빅리그 | Comedy Big League EP.402',
  'creator': '코미디빅리그',
  'creator_url': 'https://www.youtube.com/user/COMEDYBIGLEAGUE'},
 {'rank': '4',
  'content_url': 'https://www.youtube.com/watch?v=BKzdQ0lsyaM&list=PLmtapKaZsgZt3g_uAPJbsMWdkVsznn_2R&index=4',
  'content_title': "[머니올라 189화/염블

### 세부 사항 크롤링

In [16]:
content = total_contents[0]
content

{'rank': '1',
 'content_url': 'https://www.youtube.com/watch?v=hfXqgH2RscE&list=PLmtapKaZsgZt3g_uAPJbsMWdkVsznn_2R&index=1',
 'content_title': '아이유 모르는 척하기 챌린지 | ODG',
 'creator': 'odg',
 'creator_url': 'https://www.youtube.com/c/ODGkids'}

In [17]:
driver.get(content["content_url"])
js = 'document.getElementsByClassName("ytp-play-button ytp-button")[0].click()'
driver.execute_script(js)

js = 'document.getElementsByClassName("more-button style-scope ytd-video-secondary-info-renderer")[0].click()'
driver.execute_script(js)

driver.execute_script('window.scrollTo(0, 1000);')
driver.implicitly_wait(2)

In [18]:
views = driver.find_element_by_css_selector("#count > ytd-video-view-count-renderer > span.view-count.style-scope.ytd-video-view-count-renderer").text
date = driver.find_element_by_css_selector("#date > yt-formatted-string").text
good = driver.find_element_by_css_selector("#top-level-buttons > ytd-toggle-button-renderer:nth-child(1) > a").find_element_by_css_selector("#text").text
bad = driver.find_element_by_css_selector("#top-level-buttons > ytd-toggle-button-renderer:nth-child(2) > a").find_element_by_css_selector("#text").text
comments = driver.find_element_by_css_selector("#count > yt-formatted-string > span:nth-child(2)").text

In [19]:
description = driver.find_element_by_css_selector("#description")
hashlist = description.find_elements_by_css_selector("a")
texts = description.find_elements_by_css_selector("span")

hashtags = [tag.text for tag in hashlist if tag.text not in ("", " ")]        
descriptions = [span.text for span in texts if span.text not in (" ", "")]

In [20]:
views, date, good, bad, comments

('조회수 4,648,302회', '최초 공개: 2021. 4. 2.', '28만', '1.3천', '19,594')

In [13]:
hashtags

['https://www.youtube.com/1theKOriginals',
 '#NEWRELEASE',
 '#MV',
 '#1theK',
 '#원더케이',
 'https://www.youtube.com/1theK',
 'http://www.facebook.com/1theK',
 'https://twitter.com/1theK',
 'https://goo.gl/otRpZc',
 'https://vt.tiktok.com/2mSMBS']

In [14]:
descriptions

['[MV] IU(아이유)_LILAC(라일락)\n\n\nK-POP Wonderland, 1theK\n\nK-POP의 모든 즐거움을 1theK(원더케이)에서 만나보세요! :)\nWelcome to the official YouTube channel of K-POP Wonderland, 1theK\n\n""1theK Originals"" Subscribe 👉 ',
 '[Notice] 1theK YouTube is also an official channel for the MV, and music shows will count the views from this channel too.\n[공지] 1theK YouTube는 MV를 유통하는 공식 채널로, 1theK에 업로드된 MV 조회수 또한 음악방송 순위에 반영됩니다.',
 '▶1theK YT : ',
 '▶1theK FB  : ',
 '▶1theK TW : ',
 '▶1theK Kakao : ',
 '▶1theK TikTok : ',
 '"']

In [1]:
import re

def str_to_int(string : str):
    value = 0
    if '만' in string:
        value = float(re.findall('\d+\.*\d*', string)[0])
        value *= 10000
        
    elif '천' in string:
        value = float(re.findall('\d+\.*\d*', string)[0])
        value *= 1000
    else:
        tmp = re.findall('\d+\.*\d*', string)
        if len(tmp) == 0:
            value = 0
        else:
            value = float(tmp[0])
        
    return int(value)

In [2]:
def crawl_detail_content(driver, data : dict) -> dict:
    
    if data == None:
        data = {}
    
    # go to content detail page
    driver.get(data["content_url"])
    tag = driver.find_element_by_css_selector("#movie_player > div.html5-video-container > video")
    tag.click()
    
    try:
        button = driver.find_element_by_css_selector("paper-button#more")
        button.click()
    except:
        pass

    driver.execute_script('window.scrollTo(0, 1000);')
    driver.implicitly_wait(2)
    
    # get detail information of content
    views = driver.find_element_by_css_selector("#count > ytd-video-view-count-renderer > span.view-count.style-scope.ytd-video-view-count-renderer").text.replace(",", "")
    views = re.findall('\d+', views)[0]
    date = driver.find_element_by_css_selector("#date > yt-formatted-string").text.replace(".", "").split()
    date = f"{date[0]}-{date[1]}-{date[2]}"
    good = driver.find_element_by_css_selector("#top-level-buttons > ytd-toggle-button-renderer:nth-child(1) > a").find_element_by_css_selector("#text").text
    
    bad = driver.find_element_by_css_selector("#top-level-buttons > ytd-toggle-button-renderer:nth-child(2) > a").find_element_by_css_selector("#text").text
    try:
        comments = driver.find_element_by_css_selector("#count > yt-formatted-string > span:nth-child(2)").text
        comments = int(comments.replace(",", ""))
    except:
        comments = -1
    
    driver.implicitly_wait(2)
    
    # get content description and hashtags
    description = driver.find_element_by_css_selector("#description")
    hashlist = description.find_elements_by_css_selector("a")
    texts = description.find_elements_by_css_selector("span")
    hashtags = [tag.text for tag in hashlist if tag.text not in ("", " ") and tag.text.startswith("#")]        
    descriptions = [span.text for span in texts if span.text not in (" ", "")]
    
    # append detail data
    data.update(
        {
            "views" : views,
            "date" : date,
            "good" : str_to_int(good),
            "bad" : str_to_int(bad),
            "comment" : comments,
            "hashtags" : hashtags,
            "description" : descriptions
        }
    )
    
    print(data)
    
    return data
    

### 전체 크롤링

In [19]:
!pip install fake_useragent

  Created wheel for fake-useragent: filename=fake_useragent-0.1.11-cp37-none-any.whl size=13490 sha256=aed6c067c3ec92b3decbf7adc28b2604a975379951fe9859b71b423c095d08f4
  Stored in directory: C:\Users\seungsu\AppData\Local\pip\Cache\wheels\5e\63\09\d1dc15179f175357d3f5c00cbffbac37f9e8690d80545143ff
Successfully built fake-useragent


In [3]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from tqdm import tqdm
from fake_useragent import UserAgent

In [4]:
# init chrome driver
chrome_driver = "C:/Users/seungsu/Desktop/gentlegraph/crawling/chromedriver_win32/chromedriver.exe"

chrome_options = webdriver.ChromeOptions()

# chrome_options.add_argument("headless")
chrome_options.add_argument("--window-size=1100,2000")
userAgent = UserAgent().random
chrome_options.add_argument(f'user-agent={userAgent}')

prefs = {'profile.default_content_setting_values': {'cookies' : 2, 'images': 2, 'plugins' : 2, 'popups': 2, 'geolocation': 2, 'notifications' : 2, 'auto_select_certificate': 2, 'fullscreen' : 2, 'mouselock' : 2, 'mixed_script': 2, 'media_stream' : 2, 'media_stream_mic' : 2, 'media_stream_camera': 2, 'protocol_handlers' : 2, 'ppapi_broker' : 2, 'automatic_downloads': 2, 'midi_sysex' : 2, 'push_messaging' : 2, 'ssl_cert_decisions': 2, 'metro_switch_to_desktop' : 2, 'protected_media_identifier': 2, 'app_banner': 2, 'site_engagement' : 2, 'durable_storage' : 2}} 
chrome_options.add_experimental_option('prefs', prefs)
driver = webdriver.Chrome(chrome_driver, options=chrome_options)


# get playlists
driver.get("https://www.youtube.com/playlist?list=PLmtapKaZsgZt3g_uAPJbsMWdkVsznn_2R")
driver.execute_script('window.scrollTo(0, 20000);')
driver.implicitly_wait(5)
time.sleep(3)
playlist = driver.find_elements_by_css_selector("ytd-playlist-video-renderer.style-scope.ytd-playlist-video-list-renderer")

print(f"total data : {len(playlist)}")

# get brief info
total_contents = []
for content in tqdm(playlist):
    index = content.find_element_by_css_selector("#index").text
    video_info = content.find_element_by_css_selector("a#video-title")
    channel_info = content.find_element_by_css_selector("ytd-channel-name#channel-name").find_element_by_css_selector("a")
    url = video_info.get_attribute("href")
    title = video_info.text
    creator = channel_info.text
    creator_channel = channel_info.get_attribute("href")
    
    data = {
        "rank" : index,
        "content_url" : url,
        "content_title" : title,
        "creator" : creator,
        "creator_url" : creator_channel
    }
    
    total_contents.append(data)

# get detail info
for content in tqdm(total_contents):
    crawl_detail_content(driver, content)

total data : 200


  0%|                                                                                          | 0/200 [00:00<?, ?it/s]

{'rank': '1', 'content_url': 'https://www.youtube.com/watch?v=v7bnOxV4jAc&list=PLmtapKaZsgZt3g_uAPJbsMWdkVsznn_2R&index=1', 'content_title': '[MV] IU(아이유)_LILAC(라일락)', 'creator': '1theK (원더케이)', 'creator_url': 'https://www.youtube.com/c/1theK', 'views': '9407837', 'date': '2021-3-25', 'good': 860000, 'bad': 9900, 'comment': -1, 'hashtags': ['#NEWRELEASE', '#MV', '#1theK', '#원더케이'], 'description': ['[MV] IU(아이유)_LILAC(라일락)\n\n\nK-POP Wonderland, 1theK\n\nK-POP의 모든 즐거움을 1theK(원더케이)에서 만나보세요! :)\nWelcome to the official YouTube channel of K-POP Wonderland, 1theK\n\n""1theK Originals"" Subscribe 👉 ', '[Notice] 1theK YouTube is also an official channel for the MV, and music shows will count the views from this channel too.\n[공지] 1theK YouTube는 MV를 유통하는 공식 채널로, 1theK에 업로드된 MV 조회수 또한 음악방송 순위에 반영됩니다.', '▶1theK YT : ', '▶1theK FB  : ', '▶1theK TW : ', '▶1theK Kakao : ', '▶1theK TikTok : ', '"']}


  0%|▍                                                                               | 1/200 [00:22<1:14:32, 22.47s/it]

{'rank': '2', 'content_url': 'https://www.youtube.com/watch?v=6xz1bay-6dQ&list=PLmtapKaZsgZt3g_uAPJbsMWdkVsznn_2R&index=2', 'content_title': '[#하이라이트#]드디어 만남 성사! 진짜 BTS와 BTS 여고생의 만남! 살아계세요?#유퀴즈온더블럭 | YOU QUIZ ON THE BLOCK EP.99', 'creator': '유 퀴즈 온 더 튜브', 'creator_url': 'https://www.youtube.com/channel/UC920m3pMPH45qztdhppZhwA', 'views': '7676836', 'date': '2021-3-24', 'good': 460000, 'bad': 2900, 'comment': -1, 'hashtags': ['#유퀴즈온더블럭', '#유퀴즈', '#유재석', '#조세호', '#방탄소년단', '#BTS', '#RM', '#진', '#슈가', '#제이홉', '#지민', '#뷔', '#정국'], 'description': ['유 퀴즈 온 더 블럭\n\n매주 (수) 저녁 8시 40분 tvN']}


...

{'rank': '7', 'content_url': 'https://www.youtube.com/watch?v=cc__Fvyuu8g&list=PLmtapKaZsgZt3g_uAPJbsMWdkVsznn_2R&index=7', 'content_title': "[Simply K-Pop] Brave Girls (브레이브걸스) - Rollin' (롤린) _ Ep.460", 'creator': 'ARIRANG K-POP', 'creator_url': 'https://www.youtube.com/c/arirangworld', 'views': '404530', 'date': '2021-3-25', 'good': 17000, 'bad': 153, 'comment': -1, 'hashtags': ['#BraveGirls', '#Rollin', '#브레이브걸스', '#롤린'], 'description': ["Brave Girls 'Rollin''\nWith “Rollin’” released in 2017, Brave Girls is writing new history, climbing up and capturing the highest spot on the music charts. Unforgettable upon listening to it, “Rollin’” is a refreshing uptempo EDM song of the tropical house genre with an addictive chorus.\n\n브레이브걸스 '롤린'\n4년 전 발매된 브레이브걸스의 '롤린 (Rollin')'이 각종 음원 사이트에서 1위를 차지하며 역주행 신화를 새롭게 쓰고 있다. ‘롤린 (Rollin')' 은 경쾌한 업템포의 EDM 곡으로, 한번 들으면 쉽게 잊혀지지 않는 Rollin' - Rollin' - Rollin' 이라는 중독성 있는 후렴과 트로피컬 장르 특유의 재미있는 사운드가 인상적이다."]}


  4%|██▊                                                                             | 7/200 [02:19<1:01:58, 19.27s/it]

{'rank': '8', 'content_url': 'https://www.youtube.com/watch?v=W9AVhk3lqeI&list=PLmtapKaZsgZt3g_uAPJbsMWdkVsznn_2R&index=8', 'content_title': '체리가 태어났어요 | 미국 의사도 놀라게 한 감동 출산 후기', 'creator': '올리버쌤', 'creator_url': 'https://www.youtube.com/c/%EC%98%AC%EB%A6%AC%EB%B2%84%EC%8C%A4', 'views': '898285', 'date': '2021-3-26', 'good': 58000, 'bad': 196, 'comment': -1, 'hashtags': [], 'description': ['[올리버쌤의 영어회화 일력 365]\n저와 함께 새해 영어 공부를 매일 함께 해봐요!! \n교보문고 ', '예스24 ', '알라딘 ', '인터파크 ', '[해외에서 구매하고 싶다면?!]\n인터파크 글로벌 ', '드디어 체리를 만나러 산부인과에 왔어요.\n그리고 세상에서 가장 감동적인 순간을 만났어요.\n여러분과 이 특별한 순간을 공유하게 되어서 정말 기쁩니다.\n세상 모든 어머니들과 아기들에게 존경의 마음을 보내요. \n\n[올리버쌤의 실전 영어 꿀팁 100] 구매하기 \n예스24 ', '교보문고 ', '[올리버쌤의 영어 꿀팁] 책 구매하기\n예스24 ', '교보문고 ', '[올리버쌤 인스타그램] ', ' \n\n[유기견 입양 신청 및 해외 이동봉사 문의]\n메일 2798catherine@gmail.com\n인스타 instagram.com/yellow_mom_']}


  4%|███▏                                                                            | 8/200 [02:38<1:01:21, 19.18s/it]

ElementClickInterceptedException: Message: element click intercepted: Element <video tabindex="-1" class="video-stream html5-main-video" controlslist="nodownload" style="width: 640px; height: 360px; left: 0px; top: 0px;" src="blob:https://www.youtube.com/f6046329-22d5-4d78-a7a8-c92a964a38fb"></video> is not clickable at point (344, 40). Other element would receive the click: <input id="search" autocapitalize="none" autocomplete="off" autocorrect="off" name="search_query" tabindex="0" type="text" spellcheck="false" placeholder="검색" aria-label="검색">
  (Session info: chrome=89.0.4389.90)


In [17]:
content = total_contents[24]
crawl_detail_content(content["content_url"], content)

{'rank': '25', 'content_url': 'https://www.youtube.com/watch?v=yIwooGxe1dg&list=PLmtapKaZsgZt3g_uAPJbsMWdkVsznn_2R&index=25', 'content_title': '당신이 쿠지라이식 간짬뽕을 절대 알 수 없는 이유(마지막 소름 주의)', 'creator': '쿠왕coo king', 'creator_url': 'https://www.youtube.com/channel/UCR-qpXziP5pIqNHFKL-w12g', 'views': '1046553', 'date': '2021-3-22', 'good': 15000, 'bad': 1000, 'comment': -1, 'hashtags': ['#shorts'], 'description': ['당신이 쿠지라이식 간짬뽕을 알 수 없는 이유에 대해서 진지하게 알려드립니다. ', '문의 : cooking7564@gmail.com\n\n🎵 브금대통령\n🎵Track :  ']}


{'rank': '25',
 'content_url': 'https://www.youtube.com/watch?v=yIwooGxe1dg&list=PLmtapKaZsgZt3g_uAPJbsMWdkVsznn_2R&index=25',
 'content_title': '당신이 쿠지라이식 간짬뽕을 절대 알 수 없는 이유(마지막 소름 주의)',
 'creator': '쿠왕coo king',
 'creator_url': 'https://www.youtube.com/channel/UCR-qpXziP5pIqNHFKL-w12g',
 'views': '1046553',
 'date': '2021-3-22',
 'good': 15000,
 'bad': 1000,
 'comment': -1,
 'hashtags': ['#shorts'],
 'description': ['당신이 쿠지라이식 간짬뽕을 알 수 없는 이유에 대해서 진지하게 알려드립니다. ',
  '문의 : cooking7564@gmail.com\n\n🎵 브금대통령\n🎵Track :  ']}